In [ ]:
import itertools as it
from collections import Counter, defaultdict
from functools import partial
from glob import glob
from pathlib import Path

import gfapy
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.align as align
import paulssonlab.sequencing.cigar as scigar
import paulssonlab.sequencing.consensus as con
import paulssonlab.sequencing.gfa as sgfa
import paulssonlab.sequencing.io as sio
import paulssonlab.sequencing.processing as processing
from paulssonlab.util.sequence import reverse_complement

In [ ]:
hv.extension("bokeh")

In [ ]:
pl.enable_string_cache()

# Functions

In [ ]:
def concat_glob(filename):
    return pl.concat([pl.scan_ipc(f) for f in glob(filename)], how="diagonal")

In [ ]:
def load_sequencing(filename, filter=True):
    df = concat_glob(filename)
    df = df.with_columns(
        dup=pl.col("name").is_duplicated(),
        primary_alignment=pl.col("name").is_first_distinct(),
        e2e=pl.col("variants_path")
        .list.set_intersection(["<UNS9", ">UNS9", "<UNS3", ">UNS3"])
        .list.len()
        == 2,
        bc_e2e=pl.col("variants_path")
        .list.set_intersection(
            ["<BC:T7_prom", ">BC:T7_prom", "<BC:spacer2", ">BC:spacer2"]
        )
        .list.len()
        == 2,
    )
    if filter:
        df = df.filter(pl.col("primary_alignment"), pl.col("e2e"))
    return df

# Analysis

In [ ]:
%%time
df = load_sequencing(
    "/home/jqs1/scratch/sequencing/240703_pLIB476/output/max_divergence=0.3/extract_segments/*.arrow"
).collect()

In [ ]:
gfa = gfapy.Gfa.from_file(
    "/home/jqs1/scratch/sequencing/sequencing_references/pLIB473-476.gfa"
)

In [ ]:
df = df.with_columns(
    pl.col("extract_segments").struct.with_fields(
        pl.field("repressor:promoter|variant").replace({"pLacI": "pTac"})
    )
)

In [ ]:
df = processing.compute_divergences(
    df,
    list(dict.fromkeys(([s.split("=")[0] for s in gfa.segment_names]))),
    struct_name="extract_segments",
)
df2 = df.unnest("extract_segments")

In [ ]:
df2["repressor:promoter|variant"].value_counts()

In [ ]:
df2["repressor|variant"].value_counts()

In [ ]:
df2.group_by("repressor:promoter|variant", "repressor|variant").agg(pl.len()).sort(
    "len", descending=True
)

In [ ]:
df2.select(pl.col(r"^.*\|divergence$")).quantile(0.9).to_pandas().T.sort_values(0)

In [ ]:
df2 = df2.with_columns(
    max_divergence=pl.max_horizontal(
        pl.col(r"^.*\|divergence$").exclude(
            "term:DT3|divergence", "JUNC10_UNS10|divergence"
        )
    )
)

In [ ]:
plt.hist(df2["max_divergence"], bins=100, log=True);

In [ ]:
df2.filter(pl.col("max_divergence") > 0.01)

In [ ]:
df2.filter(pl.col("max_divergence") > 0.01).select(
    pl.col(r"^.*\|divergence$")
).quantile(0.9).to_pandas().T.sort_values(0)

In [ ]:
plt.hist(df2["max_divergence"], bins=100, log=True);

In [ ]:
counts = df2.select(
    pl.struct("RBS1:RBS|seq", "RBS2:RBS|seq", "RBS3:RBS|seq").alias("foo")
)["foo"].value_counts(sort=True)

In [ ]:
counts["count"].to_pandas().hvplot.step(logy=True, logx=True)

In [ ]:
df["grouping_depth"].sort(descending=True).to_pandas().hvplot.step(logy=True)

In [ ]:
counts.filter(pl.col("count") == 1)

In [ ]:
counts.filter(pl.col("count") > 5)

In [ ]:
with pl.Config(tbl_rows=100):
    display(counts)

In [ ]:
perfect_segments = [
    "pBetI",
    "RBS1:upstream",
    "LacI",
    "pTac",
    "RBS2:upstream",
    "PhlF",
    "pPhlF",
    "RBS3:upstream",
    "BetI",
]

In [ ]:
df2.filter(*[pl.col(f"{s}|divergence") == 0 for s in perfect_segments]).select(
    pl.struct("RBS1:RBS|seq", "RBS2:RBS|seq", "RBS3:RBS|seq").alias("foo")
)["foo"].value_counts(sort=True)

In [ ]:
filter_columns = [
    "promoter|divergence",
    "ScmJ|divergence",
    "RBS:RiboJ|divergence",
    "RBS:BCD_leader|divergence",
    "repressor:promoter|divergence",
    "repressor|divergence",
    "mVenus|divergence",
]

In [ ]:
157_002 / len(df2)

In [ ]:
df2.filter(*[pl.col(c) == 0 for c in filter_columns])

# 240703_pLIB476 vs. 240703_pLIB476_bottlenecked

In [ ]:
%%time
df_240703b = load_sequencing(
    "/home/jqs1/scratch/sequencing/240703_pLIB476_bottlenecked/output/max_divergence=0.3/extract_segments/*.arrow"
).collect()
df_240703 = load_sequencing(
    "/home/jqs1/scratch/sequencing/240703_pLIB476/output/max_divergence=0.3/extract_segments/*.arrow"
).collect()

In [ ]:
%%time
cols = ["grouping_path", "variants_path", "grouping_depth"]
df_joined = df_240703.select(cols).join(
    df_240703b.select(cols),
    on=processing.categorical_list_hash(pl.col("grouping_path")),
    # on="grouping_path_hash",
    how="full",
    suffix="_b",
)

In [ ]:
df_joined.group_by(
    pl.col("grouping_path").is_not_null(), pl.col("grouping_path_b").is_not_null()
).agg(pl.len()).rename(
    {
        "grouping_path": "240703",
        "grouping_path_b": "240703_bottlenecked",
    }
)

In [ ]:
df_joined.filter(
    pl.col("grouping_path_b").is_not_null(), pl.col("grouping_path").is_null()
)["grouping_depth_b"].sort(descending=True)

In [ ]:
df_240703b["grouping_depth"].quantile(0.999)

In [ ]:
df_240703b["grouping_depth"].sort(descending=True).to_pandas().hvplot.step(logy=True)

In [ ]:
df_240703["grouping_depth"].sort(descending=True).to_pandas().hvplot.step(logy=True)

In [ ]:
df_240703b.sort("grouping_depth").select(
    pl.col("grouping_depth"),
    frac_barcodes=pl.int_range(1, pl.len() + 1, dtype=pl.UInt32) / pl.len(),
    frac_reads=pl.col("grouping_depth").cum_sum() / pl.col("grouping_depth").sum(),
).to_pandas().hvplot.step("grouping_depth", logx=True, logy=False, where="pre")

In [ ]:
df_240703.sort("grouping_depth").select(
    pl.col("grouping_depth"),
    frac_barcodes=pl.int_range(1, pl.len() + 1, dtype=pl.UInt32) / pl.len(),
    frac_reads=pl.col("grouping_depth").cum_sum() / pl.col("grouping_depth").sum(),
).to_pandas().hvplot.step("grouping_depth", logx=True, logy=False, where="pre")

In [ ]:
df_joined.select("grouping_depth", "grouping_depth_b").to_pandas().hvplot.scatter(
    "grouping_depth",
    "grouping_depth_b",
    s=0.8,
    alpha=0.5,
)